# CrewAI Tutorial: Building Multi-Agent AI Systems

## What is CrewAI?

CrewAI is an open-source Python framework designed to orchestrate role-playing, autonomous AI agents that collaborate to tackle complex tasks. Think of it as building a team where each AI agent has specialized skills and responsibilities.

## Core Concepts

### 1. **Agents**
Think of an agent as a specialized team member with specific skills, expertise, and responsibilities. For example, a Researcher agent might excel at gathering and analyzing information, while a Writer agent might be better at creating content.

### 2. **Tasks**
Tasks are specific assignments given to agents. Each task defines what needs to be accomplished, expected outcomes, and which agent should handle it.

### 3. **Crews**
A crew is a structured group of agents and tasks that work together to complete a process. Crews orchestrate the collaboration between multiple agents.

### 4. **Tools**
This enables agents to perform tasks beyond their core capabilities, such as retrieving information from the web or performing complex data analysis.

## Tutorial Overview

We'll build a **Content Research and Writing System** with:
- **Research Agent**: Gathers information on topics
- **Writer Agent**: Creates articles based on research
- **Editor Agent**: Reviews and improves content

This tutorial demonstrates the power of multi-AI agent systems that exceed the performance of prompting a single LLM by designing and prompting a team of AI agents.

## Step 1: Setup and Environment Configuration

In [ ]:
# Install required packages
!pip install crewai crewai-tools python-dotenv

In [ ]:
# Load environment variables
import os
from dotenv import load_dotenv

# Load API keys from .env file
load_dotenv()

# Required environment variables:
# OPENAI_API_KEY=your_openai_key_here
# SERPER_API_KEY=your_serper_key_here (for web search)

print("Environment variables loaded successfully!")
print(f"OpenAI API Key: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Missing'}")
print(f"Serper API Key: {'✓ Set' if os.getenv('SERPER_API_KEY') else '✗ Missing'}")

## Step 2: Import CrewAI Components

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, FileWriterTool
from langchain_openai import ChatOpenAI

## Step 3: Initialize Tools and LLM

In [ ]:
# Initialize tools
search_tool = SerperDevTool()
file_writer = FileWriterTool()

# Initialize LLM
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.1
)

print("Tools and LLM initialized successfully!")

## Step 4: Create Specialized Agents

In [ ]:
# Research Agent - Expert at gathering information
researcher = Agent(
    role='Senior Research Analyst',
    goal='Conduct thorough research on given topics and provide comprehensive insights',
    backstory="""You are an experienced research analyst with expertise in gathering, 
    analyzing, and synthesizing information from multiple sources. You have a keen eye 
    for detail and excel at identifying key trends and insights.""",
    tools=[search_tool],
    llm=llm,
    verbose=True
)

# Writer Agent - Expert at content creation
writer = Agent(
    role='Content Writer',
    goal='Create engaging, informative articles based on research findings',
    backstory="""You are a skilled content writer with years of experience in creating 
    compelling articles. You excel at transforming complex research into accessible, 
    engaging content that resonates with readers.""",
    tools=[file_writer],
    llm=llm,
    verbose=True
)

# Editor Agent - Expert at content review and improvement
editor = Agent(
    role='Senior Editor',
    goal='Review and enhance content for clarity, accuracy, and engagement',
    backstory="""You are a meticulous editor with exceptional attention to detail. 
    You excel at improving content structure, clarity, and overall quality while 
    maintaining the author's voice and intent.""",
    llm=llm,
    verbose=True
)

print("Agents created successfully!")

## Step 5: Define Tasks for Each Agent

In [ ]:
def create_tasks(topic):
    # Research Task
    research_task = Task(
        description=f"""Conduct comprehensive research on: {topic}
        
        Your research should include:
        - Current trends and developments
        - Key statistics and data points
        - Expert opinions and insights
        - Future implications and predictions
        
        Provide a detailed research summary with sources.""",
        agent=researcher,
        expected_output="A comprehensive research summary with key findings and sources"
    )
    
    # Writing Task
    writing_task = Task(
        description=f"""Based on the research findings, write an engaging article about: {topic}
        
        The article should:
        - Be 800-1000 words long
        - Have a compelling introduction and conclusion
        - Include relevant statistics and expert insights
        - Be structured with clear headings and subheadings
        - Be written for a general audience
        
        Save the article as '{topic.lower().replace(' ', '_')}_article.md'""",
        agent=writer,
        context=[research_task],
        expected_output="A well-structured, engaging article saved as a markdown file"
    )
    
    # Editing Task
    editing_task = Task(
        description="""Review the written article and provide:
        
        1. Overall assessment of content quality
        2. Suggestions for improvement in:
           - Structure and flow
           - Clarity and readability
           - Factual accuracy
           - Engagement and interest
        3. A final edited version with improvements
        
        Provide both your review comments and the improved article.""",
        agent=editor,
        context=[writing_task],
        expected_output="Editorial review with improvement suggestions and final edited article"
    )
    
    return [research_task, writing_task, editing_task]

print("Task creation function defined!")

## Step 6: Create and Execute the Crew

In [ ]:
# Define the topic for research and writing
topic = "Artificial Intelligence in Healthcare 2024"

# Create tasks for the topic
tasks = create_tasks(topic)

# Create the crew
content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=tasks,
    process=Process.sequential,  # Tasks executed one after another
    verbose=2  # Detailed logging
)

print(f"Crew created successfully for topic: {topic}")
print(f"Number of agents: {len(content_crew.agents)}")
print(f"Number of tasks: {len(content_crew.tasks)}")

In [ ]:
# Execute the crew
print("Starting crew execution...\n")

result = content_crew.kickoff()

print("\n" + "="*50)
print("CREW EXECUTION COMPLETED!")
print("="*50)
print("\nFinal Result:")
print(result)

## Step 7: Advanced Features and Best Practices

In [ ]:
# Advanced Crew Configuration Example
def create_advanced_crew(topic):
    """Create a crew with advanced features"""
    
    # Enhanced agents with memory and delegation
    senior_researcher = Agent(
        role='Senior Research Director',
        goal=f'Lead comprehensive research on {topic}',
        backstory="You're a research director with 15+ years of experience",
        tools=[search_tool],
        llm=llm,
        allow_delegation=True,  # Can delegate tasks to other agents
        memory=True,  # Remembers previous interactions
        max_iter=3,  # Maximum iterations for task completion
        verbose=True
    )
    
    # Create crew with custom configuration
    advanced_crew = Crew(
        agents=[senior_researcher, writer, editor],
        tasks=create_tasks(topic),
        process=Process.sequential,
        memory=True,  # Crew-level memory
        cache=True,   # Cache results for efficiency
        max_rpm=10,   # Rate limiting
        verbose=2
    )
    
    return advanced_crew

print("Advanced crew configuration example defined!")

# Best Practices Summary
best_practices = """
🚀 CrewAI Best Practices:

1. **Agent Design**: Give agents specific roles and detailed backstories
2. **Task Dependencies**: Use context parameter to chain tasks effectively
3. **Tool Selection**: Equip agents with relevant tools for their roles
4. **Process Types**: Choose sequential for dependent tasks, hierarchical for complex workflows
5. **Error Handling**: Set max_iter and implement validation in your tasks
6. **Performance**: Use caching and memory for better efficiency
7. **Monitoring**: Enable verbose logging to track agent interactions
"""

print(best_practices)

## Conclusion

You've successfully built a multi-agent system using CrewAI! This tutorial covered:

✅ **Environment Setup**: API keys and dependencies  
✅ **Agent Creation**: Specialized AI agents with distinct roles  
✅ **Task Definition**: Clear objectives and expected outputs  
✅ **Crew Orchestration**: Coordinating agents to work together  
✅ **Execution**: Running the complete workflow  
✅ **Advanced Features**: Memory, delegation, and optimization  

## Next Steps

- Explore [CrewAI Tools](https://docs.crewai.com/concepts/tools) for additional capabilities
- Learn about [Hierarchical Process](https://docs.crewai.com/concepts/processes) for complex workflows
- Try [CrewAI Studio](https://app.crewai.com) for visual crew building
- Check out the [GitHub repository](https://github.com/crewAIInc/crewAI) for more examples

**Remember**: Learn key principles of designing effective AI agents, and organizing a team of AI agents to perform complex, multi-step tasks is the foundation of successful multi-agent systems!